# The Cholesky decomposition

In [3]:
import numba
import numpy as np
from numba import njit

In [4]:
xvals = np.linspace(-3, 3, 100)

Let ${\bf \Sigma}$ be an $N\,P\times N\,P$ positive definite matrix such that
$$
{\bf \Sigma} =
\begin{bmatrix}
    {\bf \sigma}_{1,1} & {\bf \sigma}_{1,2} & \cdots & {\bf \sigma}_{1,N}\\
    {\bf \sigma}_{2,1} & {\bf \sigma}_{2,2} & \cdots & {\bf \sigma}_{2,N}\\
    \vdots & \vdots & \ddots & \vdots \\
    {\bf \sigma}_{N,1} & {\bf \sigma}_{N,2} & \ldots & {\bf \sigma}_{N,N}
\end{bmatrix}
$$

We seek to decompose ${\bf \Sigma} = {\bf L\,R\,L}^\intercal$
with ${\bf R} = {\rm diag}(R_1, \ldots, R_N)$ and
${\bf L} = \{L_{i,j} : i \in \{1, \ldots N\}, j \leq i\}$ a block-lower triangular matrix with $L_{i,i} = {\bf I}$

We seek ${\bf L}$ and ${\bf R}$ such that
$$
    ({\bf L\,R\,L}^\intercal)_{i,j} = \sigma_{i,j}
$$

The choleskly decomposition yields the following algorithm

$$
\begin{aligned}
    {\bf R}_i &= \sigma_{i,i} - \sum_{k=1}^{i-1}{\bf L}_{i,k}\,{\bf R}_{k}\,{\bf L}_{i,k}^\intercal
    & \text{for }i=1,\ldots,T\\
    {\bf L}_{i,j} &= \left[\sigma_{i,j} - \sum_{k=1}^{j-1}{\bf L}_{i,k}\,{\bf R}_k\,{\bf L}_{j,k}^\intercal\right]
    &\text{for } i=2,\ldots,T;\, j<i
\end{aligned}
$$

In [15]:
d = 10
S = np.random.randn(d, d)
S = S @ S.T

In [16]:
LR = np.linalg.cholesky(S)

In [23]:
np.set_printoptions(suppress=True, precision=5, linewidth=150)

In [29]:
from scipy.linalg import cholesky

In [49]:
S

array([[ 5.67853,  0.63172,  0.97465,  3.41199,  0.16787, -1.16823, -1.96392,  1.21175,  1.97658, -1.23348],
       [ 0.63172,  6.2311 ,  0.48945,  3.25172,  4.09169, -0.44712,  2.15596,  3.09186,  0.47704,  3.34399],
       [ 0.97465,  0.48945, 21.27481, -4.28209, -6.14287, -4.06289, -1.03766, -6.35409, -2.84032,  4.00541],
       [ 3.41199,  3.25172, -4.28209,  7.10469,  4.08296, -0.88207,  0.87026,  6.93269, -0.01353, -0.68472],
       [ 0.16787,  4.09169, -6.14287,  4.08296, 10.30687, -3.36179,  3.26869,  3.76894,  0.94472,  5.99554],
       [-1.16823, -0.44712, -4.06289, -0.88207, -3.36179, 10.21128, -2.34882, -0.77209,  6.11035, -1.83889],
       [-1.96392,  2.15596, -1.03766,  0.87026,  3.26869, -2.34882,  5.78679,  3.60771, -2.78598, -0.32747],
       [ 1.21175,  3.09186, -6.35409,  6.93269,  3.76894, -0.77209,  3.60771, 20.07348, -1.7007 , -7.22475],
       [ 1.97658,  0.47704, -2.84032, -0.01353,  0.94472,  6.11035, -2.78598, -1.7007 ,  8.12   ,  2.17052],
       [-1.23348,  

In [50]:
LR

array([[ 2.38297,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.2651 ,  2.4821 ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.40901,  0.15351,  4.59173,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 1.43182,  1.15715, -1.09879,  1.58374,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.07045,  1.64096, -1.39895,  0.34483,  2.35228,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [-0.49024, -0.12778, -0.83689, -0.601  , -1.73495,  2.42548,  0.     ,  0.     ,  0.     ,  0.     ],
       [-0.82415,  0.95663, -0.18456,  0.4676 ,  0.56861, -0.62566,  1.79582,  0.     ,  0.     ,  0.     ],
       [ 0.50851,  1.19135, -1.46894,  2.0281 , -0.41498, -0.45392,  0.9019 ,  3.3065 ,  0.     ,  0.     ],
       [ 0.82946,  0.1036 , -0.69592, -1.31696,  0.08368,  2.18575, -0.2195 ,  0.18981,  0.57781,  0.     ],
       [-0.51762,  

In [65]:
np.diag(LR) ** 2

array([ 5.67853,  6.16083, 21.08396,  2.50824,  5.53322,  5.88298,  3.22496, 10.93296,  0.33386,  0.95383])

In [91]:
R_vals = np.zeros(d)
L_vals = np.zeros((d,d))

## Step 1

In [92]:
R_vals[0] = S[0,0]
R_vals[0]

5.678527296717118

In [94]:
R_vals

array([5.67853, 0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ])

In [93]:
L_vals[:, 0] = S[:, 0] / R_vals[0]
L_vals[:, 0]

array([ 1.     ,  0.11125,  0.17164,  0.60086,  0.02956, -0.20573, -0.34585,  0.21339,  0.34808, -0.21722])

In [73]:
L_vals

array([[ 1.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.11125,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.17164,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.60086,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.02956,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [-0.20573,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [-0.34585,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.21339,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.34808,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [-0.21722,  

## Step 2

In [78]:
L_vals[1,0]

0.1112467918769485

In [83]:
R_vals[1] = S[1,1] - L_vals[1,0] * R_vals[0] * L_vals[1,0]
R_vals[1]

6.160825443886867

In [87]:
L_vals[1:, 1] = (S[1:,1] - L_vals[1:,0] * R_vals[0] * L_vals[1,0]) / R_vals[1]

In [88]:
L_vals

array([[ 1.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.11125,  1.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.17164,  0.06185,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.60086,  0.4662 ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.02956,  0.66112,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [-0.20573, -0.05148,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [-0.34585,  0.38541,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.21339,  0.47998,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.34808,  0.04174,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [-0.21722,  